### El Perceptrón y su implementación con Keras

Un perceptrón clásico es una neurona artificial que calcula una combinación lineal de las entradas y luego aplica una función de activación (por ejemplo, escalón o sigmoide).

En Keras podemos representar un perceptrón binario como un modelo `Sequential` con una sola capa `Dense` de una neurona y activación sigmoide, entrenado con `binary_crossentropy`.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf

## Perceptrón en datos de estudiantes

Trabajaremos con un dataset de notas de estudiantes vs. horas de estudio. La variable objetivo indicará si el estudiante **aprueba** (1) o **no aprueba** (0).

In [ ]:
# Carga del dataset de estudiantes
df = pd.read_csv('student_scores.csv')
df.head()

### Creación de la variable objetivo

Supondremos que la nota de aprobación es 55. Creamos la columna `aprueba` que vale 1 si la nota es mayor o igual a 55 y 0 en caso contrario.

In [ ]:
df['aprueba'] = (df['Scores'] >= 55).astype(int)
df.head()

### Definición de X e y

Usaremos solamente la columna `Hours` como feature y la columna `aprueba` como etiqueta.

In [ ]:
X = df[['Hours']].values
y = df['aprueba'].values
X.shape, y.shape

### División train/test

Dividimos el dataset en entrenamiento y prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

### Definición del perceptrón con Keras

El modelo será una red `Sequential` con una sola neurona de salida y activación sigmoide.
Usaremos `binary_crossentropy` como función de pérdida y mediremos `accuracy`.

In [ ]:
tf.random.set_seed(42)

model_student = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,), activation='sigmoid')
])

model_student.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_student.summary()

### Entrenamiento del perceptrón (estudiantes)

In [ ]:
history_student = model_student.fit(
    X_train, y_train,
    epochs=100,
    batch_size=8,
    validation_split=0.2,
    verbose=0
)

len(history_student.history['loss'])

### Predicciones y evaluación (estudiantes)

In [ ]:
# Predicciones en el set de test
y_test_prob = model_student.predict(X_test).flatten()
y_test_pred = (y_test_prob >= 0.5).astype(int)

acc_student = accuracy_score(y_test, y_test_pred)
cm_student = confusion_matrix(y_test, y_test_pred)

print('Accuracy (estudiantes):', acc_student)
print('Matriz de confusión (estudiantes):')
print(cm_student)

In [ ]:
# Predicción para un estudiante que estudia 5 horas
hours_example = np.array([[5]])
prob_5h = model_student.predict(hours_example)[0, 0]
pred_5h = int(prob_5h >= 0.5)

print('Probabilidad de aprobar (5 horas):', prob_5h)
print('Aprueba (1) o no (0):', pred_5h)

### Evolución del error (estudiantes)

In [ ]:
loss_st = history_student.history['loss']
val_loss_st = history_student.history['val_loss']
epochs_st = range(1, len(loss_st) + 1)

plt.figure(figsize=(8,4))
plt.plot(epochs_st, loss_st, label='Entrenamiento')
plt.plot(epochs_st, val_loss_st, label='Validación')
plt.xlabel('Epoch')
plt.ylabel('Binary crossentropy')
plt.title('Evolución del error (estudiantes)')
plt.legend()
plt.show()

## Perceptrón aplicado al Titanic

Ahora aplicaremos un modelo similar al clásico problema del Titanic, donde el objetivo es predecir si una persona sobrevivió (`Survived`).

In [ ]:
# Carga del dataset Titanic
df_titanic = pd.read_csv('titanic.csv')
df_titanic.head()

### Limpieza de datos

Eliminamos filas con valores nulos en las columnas que usaremos en el modelo para simplificar el preprocesamiento.

In [ ]:
cols_model = ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
df_titanic_clean = df_titanic[cols_model].dropna().reset_index(drop=True)
df_titanic_clean.head()

### Definición de X e y (Titanic)

Usaremos algunas columnas como features y `Survived` como variable objetivo.

In [ ]:
X_titanic = df_titanic_clean.drop(columns='Survived')
y_titanic = df_titanic_clean['Survived'].values

X_titanic.head()

### Binarización y escalamiento (Titanic)

- Binarizamos las columnas categóricas (`Sex`, `Embarked`) usando `get_dummies`.
- Escalamos las columnas numéricas usando `StandardScaler`.

In [ ]:
# Binarización de variables categóricas
X_titanic_enc = pd.get_dummies(X_titanic, columns=['Sex', 'Embarked'], drop_first=True)
X_titanic_enc.head()

In [ ]:
# Escalamiento de los datos
scaler_titanic = StandardScaler()
X_titanic_scaled = scaler_titanic.fit_transform(X_titanic_enc)
X_titanic_scaled.shape

### División train/test (Titanic)

In [ ]:
X_tr_t, X_te_t, y_tr_t, y_te_t = train_test_split(
    X_titanic_scaled, y_titanic, test_size=0.2, random_state=42, stratify=y_titanic
)

X_tr_t.shape, X_te_t.shape

### Definición del perceptrón Keras (Titanic)

Usamos nuevamente una sola neurona de salida con activación sigmoide para clasificación binaria.

In [ ]:
tf.random.set_seed(42)

model_titanic = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(X_tr_t.shape[1],), activation='sigmoid')
])

model_titanic.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_titanic.summary()

### Entrenamiento del modelo (Titanic)

In [ ]:
history_titanic = model_titanic.fit(
    X_tr_t, y_tr_t,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

len(history_titanic.history['loss'])

### Evaluación del modelo (Titanic)

In [ ]:
y_te_prob_t = model_titanic.predict(X_te_t).flatten()
y_te_pred_t = (y_te_prob_t >= 0.5).astype(int)

acc_titanic = accuracy_score(y_te_t, y_te_pred_t)
cm_titanic = confusion_matrix(y_te_t, y_te_pred_t)

print('Accuracy (Titanic):', acc_titanic)
print('Matriz de confusión (Titanic):')
print(cm_titanic)

### Evolución del error (Titanic)

In [ ]:
loss_t = history_titanic.history['loss']
val_loss_t = history_titanic.history['val_loss']
epochs_t = range(1, len(loss_t) + 1)

plt.figure(figsize=(8,4))
plt.plot(epochs_t, loss_t, label='Entrenamiento')
plt.plot(epochs_t, val_loss_t, label='Validación')
plt.xlabel('Epoch')
plt.ylabel('Binary crossentropy')
plt.title('Evolución del error (Titanic)')
plt.legend()
plt.show()

### Comentarios y conclusiones

- Usar TensorFlow/Keras nos permite definir y entrenar perceptrones de forma muy concisa.
- En el caso de estudiantes, el problema es casi lineal y el modelo logra alta exactitud.
- En el Titanic, el problema es más complejo; un solo perceptrón puede no capturar toda la estructura de los datos, pero sirve como línea base.
- A partir de aquí, se pueden extender los modelos agregando capas ocultas o probando otros optimizadores y tasas de aprendizaje.